In [41]:
# Importacion de librerias
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.io import arff
from sklearn.model_selection import KFold

In [42]:
def readData(rutaArchivo, formato='csv'):
    if formato == 'csv':
        data = pd.read_csv(rutaArchivo)
    elif formato == 'arff':
        data = arff.loadarff(rutaArchivo)
        data = pd.DataFrame(data[0])
    return data

def getNombreClases(data, nombreClase, formato='csv'):
    nombreClases = data[nombreClase].unique()
    if formato == 'arff':
        nombreClases = [clase.decode('utf-8') if isinstance(clase, bytes) else clase for clase in nombreClases]
    return nombreClases

In [80]:
def getFrequencyCounts(data, clases):
    frequency_counts = {}
    indices = {}

    for i, (x_val, clase_val) in enumerate(zip(data.iloc[:, 0], data.iloc[:, -1])):
        if x_val not in frequency_counts:
            frequency_counts[x_val] = {
                'frecuencias': {cl: 0 for cl in clases},
                'indices': []
                                       }

        frequency_counts[x_val][clase_val] += 1
        frequency_counts[x_val].append(i)

    # frequency_counts = {
    #     x: {
    #         'frecuencias': {c: frequency_counts[x]['frecuencias'][c] for c in sorted(frequency_counts[x]['frecuencias'])},
    #         'indices': frequency_counts[x]['indices']
    #     }
    #     for x in sorted(frequency_counts)
    # }

    frequency_counts = {
        x: {c: frequency_counts[x][c] for c in sorted(frequency_counts[x])}
        for x in sorted(frequency_counts)
    }

    indices = {x: indices[x] for x in sorted(indices)}

    resultado = {
        "frecuencias": frequency_counts,
        "indices": indices
    }

    return resultado

def getFrequencyCount_Mayor(data, promedios, clases):
    frequency_counts = {}

    for umbral in promedios:
        # Ramas del split
        df_si  = data[data.iloc[:, 0] <= umbral]   # Rama 'Sí'
        df_no  = data[data.iloc[:, 0] >  umbral]   # Rama 'No'

        # Conteos por clase en cada rama
        conteos_si = df_si[data.columns[-1]].value_counts().to_dict()
        conteos_no = df_no[data.columns[-1]].value_counts().to_dict()

        # Asegurar que todas las clases aparezcan aunque sea con 0
        rama_si = {cl: conteos_si.get(cl, 0) for cl in clases}
        rama_no = {cl: conteos_no.get(cl, 0) for cl in clases}

        # Agregar los índices de las filas incluidas
        indices_si = df_si.index.tolist()
        indices_no = df_no.index.tolist()

        # frequency_counts[umbral] = {
        #     'No': {
        #         'frecuencias': rama_no,
        #         'indices': indices_no
        #     },
        #     'Sí': {
        #         'frecuencias': rama_si,
        #         'indices': indices_si
        #     }
        # }

        frequency_counts[umbral] = {
            'frecuencias': {
                'No': rama_no,
                'Sí': rama_si
            },
            'indices': {
                'No': indices_no,
                'Sí': indices_si
            }
        }

    return frequency_counts
                

        

In [44]:
def getMeans(data):
    dataSorted = data.sort_values(by=data.columns[0])
    # print(dataSorted)
    # Tomar promedios entre los valores por cada par de filas consecutivas
    means = []
    for i in range(len(dataSorted) - 1):
        val1 = dataSorted.iloc[i, 0]
        val2 = dataSorted.iloc[i + 1, 0]
        mean = (val1 + val2) / 2
        means.append(float(mean))
    # print("Means:", means)

    return means

In [45]:
def gini_columna(counts):
    """
    Calcula la impureza de Gini para cada hoja.
    Parámetros:
    @data: DataFrame de pandas que contiene la frecuencia de cada clase en una hoja.

    Retorna:
    @gini: Valor de la impureza de Gini para la hoja.
    """
    total = counts.sum()
    p = counts / total

    return 1 - np.sum(p ** 2)

def getImpurezaGini(data):
    """
    Calcula la impureza de Gini para un conjunto de datos dado.
    Parámetros:
    @data: DataFrame de pandas que contiene la frecuencia de cada clase.

    Retorna:
    @sumaPonderadaGini: Valor de la impureza de Gini para el atributo.
    """

    gini_por_hoja = data.apply(gini_columna, axis=0)

    # --- Pesos (proporción de elementos por hoja) ---
    tam = data.sum(axis=0)   # total por hoja
    total = tam.sum()

    pesos = tam / total

    # Calculo de la Gini ponderada
    gini_ponderada = (pesos * gini_por_hoja).sum()

    return gini_por_hoja, gini_ponderada

In [82]:
def calculateGiniImpurity(data, clasesNombre, nAtributos):
    clase = data.columns[-1]
    giniAtributo = []
    
    # Creacion de nuevos dataframes por atributo, con la columna de clase
    for atributo in nAtributos:
        frecuencias = {}
        df_fq = pd.DataFrame()

        nuevo_df = pd.DataFrame({
            atributo: data[atributo],
            "Clase": data[clase]
        })

        # Detectando si es un dato categórico o numérico
        if pd.api.types.is_numeric_dtype(nuevo_df[atributo]):
            promedios = getMeans(nuevo_df)
            frecuencias = getFrequencyCount_Mayor(nuevo_df, promedios, clasesNombre)
            print("Frecuencias atributo ", atributo, ": ", frecuencias)
            # Obtener impureza de Gini por cada promedio (umbral)
            giniUmbrales = []
            for umbral, conteos in frecuencias.items():
                df_fq = pd.DataFrame(conteos['frecuencias'])
                giniHojas, gini = getImpurezaGini(df_fq)
                giniUmbrales.append((umbral, float(gini)))

            menorGini = min(giniUmbrales, key=lambda x: x[1])

            giniAtributo.append((atributo, float(menorGini[1]), menorGini[0] ))
            
            
        else:
            print("df: \n", nuevo_df)
            frecuencias = getFrequencyCounts(nuevo_df, clasesNombre)
            print("Frecuencias atributo ", atributo, ": ", frecuencias)
            df_fq = pd.DataFrame(frecuencias['frecuencias'])
            giniHojas, gini = getImpurezaGini(df_fq)

            giniAtributo.append((atributo, float(gini)))

    return giniAtributo


In [47]:
def createDecisionTree(data, clasesNombre, nAtributos):
    giniAtributo = calculateGiniImpurity(data, clasesNombre, nAtributos)

    # Seleccionar el atributo con la menor impureza de Gini
    n0 = min(giniAtributo, key=lambda x: x[1])
    

    return n0

In [83]:
# Ejemplo


gustoCine = {
    "Id": [1, 2, 3, 4, 5, 6, 7],
    "Palomitas": ["Sí", "Sí", "No", "No", "Sí", "Sí", "No"],
    "Malteada": ["Sí", "No", "Sí", "Sí", "Sí", "No", "No"],
    "Edad": [7, 12, 18, 35, 38, 50, 83],
    "Cine": ["No", "No", "Sí", "Sí", "Sí", "No", "No"]   
}

data = pd.DataFrame(gustoCine)
c = 1
# print(data.head())

# print("--------------------")
# print(data.columns[c])

# print("********************")
# print(data.iloc[:, c].dtype)

# print("********************")
# print(data.iloc[:, c])

# print("--------------------")

clasesNombre = getNombreClases(data, "Cine", formato='csv')
print("Clases: ", clasesNombre)

data.drop(columns=["Id"], inplace=True)

nAtributos = data.columns[:-1]
# clase = data.columns[-1]
# giniAtributo = []
   
giniAtributo = calculateGiniImpurity(data, clasesNombre, nAtributos)
# win = min(giniAtributo, key=lambda x: x[1])
    
    # print("Impureza Gini: ", gini)
print("Gini Atributo: ", giniAtributo)
# print("Mejor Atributo: ", win)

Clases:  ['No' 'Sí']
df: 
   Palomitas Clase
0        Sí    No
1        Sí    No
2        No    Sí
3        No    Sí
4        Sí    Sí
5        Sí    No
6        No    No


KeyError: 'No'

In [ ]:
# Main
data = readData('iris.csv', formato='csv')
c = 1
print(data.head())

print("--------------------")
print(data.columns[c])

print("********************")
print(data.iloc[:, c].dtype)

print("********************")
print(data.iloc[:, c])

print("--------------------")

clasesNombre = getNombreClases(data, "Species", formato='csv')
print("Clases: ", clasesNombre)




# gini = getImpurezaGini(data.iloc[:, c], clasesNombre)
# print(gini)

   Id  SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm      Species
0   1            5.1           3.5            1.4           0.2  Iris-setosa
1   2            4.9           3.0            1.4           0.2  Iris-setosa
2   3            4.7           3.2            1.3           0.2  Iris-setosa
3   4            4.6           3.1            1.5           0.2  Iris-setosa
4   5            5.0           3.6            1.4           0.2  Iris-setosa
--------------------
SepalLengthCm
********************
float64
********************
0      5.1
1      4.9
2      4.7
3      4.6
4      5.0
      ... 
145    6.7
146    6.3
147    6.5
148    6.2
149    5.9
Name: SepalLengthCm, Length: 150, dtype: float64
--------------------
Clases:  ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
